# Self-reflection pattern

Self-reflection pattern contains at least 2 promtps: the prompt that perform task itself and prompt to reflect the previous response. In self-reflection pattern, 2 prompt are performed by same LLM model in separated fashion.

In this example, we are using llama index API and an ollama model hosted locally.

## Setup
host an ollama instance by using either running a docker or a run a service following instruction from ollama's guide.

In [1]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import AgentMessage
from aap_core.orchestration import ReflectionAgent
from aap_llamaindex.chain import ChatCausalMultiTurnsChain
from llama_index.llms.ollama import Ollama

In [2]:
model = Ollama(model="qwen3:4b-thinking-2507-q4_K_M", base_url="192.168.55.1:11434", request_timeout=400, context_window=32768)

In [ ]:
system_prompt_task = """/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is too long (for example greater than 80 lines), split the logic into multiple smaller functions.
- All functions should have docstring explanation. In the explanation, there should be an simple example to illustrate the function and how to call it.
- The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code
"""
user_prompt_task = "{query}"
task_chain = ChatCausalMultiTurnsChain(
    model=model,
    system_prompt=system_prompt_task,
    user_prompt_template=user_prompt_task,
    base_url="192.168.55.1:11434",
    thinking=False,
    request_timeout=400, context_window=32768)
task_chain.final_response_as_context("response")

system_prompt_reflection = """/no_think You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain full function code, inline code comment and docstring, nothing else."""
user_prompt_reflection = "Input query: {query}\n\nFunction implementation: {context_response}"
# Use the same llm for task and self-reflection
reflection_chain = ChatCausalMultiTurnsChain(
    model=model,
    system_prompt=system_prompt_reflection,
    user_prompt_template=user_prompt_reflection,
    base_url="192.168.55.1:11434",
    thinking=False,
    request_timeout=400, context_window=32768)  # use the same model for self-reflection

def state_callback(state: str):
    print(f"agent state: {state}")
reflection_skill = AgentSkill(
    id='reflection-skill',
    name="reflection skill",
    description="self-reflection skill",
    tags=['reflection']
)
reflection_card = AgentCard(
    name="reflection agent",
    description="self-reflection agent",
    skills=[reflection_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
reflection_agent = ReflectionAgent(card=reflection_card, chain_task=task_chain, chain_reflection=reflection_chain, state_change_callback=state_callback)

In [4]:
# Take a leetcode as an example. Source: https://leetcode.com/problems/palindrome-partitioning-ii/description/
query = """Write python function(s) to solve the following problem:
Given a string s, partition s such that every substring of the partition is a palindrome.
Return the minimum cuts needed for a palindrome partitioning of s.

Example 1:
Input: s = "aab"
Output: 1
Explanation: The palindrome partitioning ["aa","b"] could be produced using 1 cut.

Example 2:
Input: s = "a"
Output: 0

Example 3:
Input: s = "ab"
Output: 1

Constraints:
1 <= s.length <= 2000
s consists of lowercase English letters only."""

final_message = reflection_agent.execute(AgentMessage(query=query))
for response in final_message.responses:
    name, msg = response
    print(name)
    print(msg)

agent state: reflection agent:running
agent state: reflection agent:reflecting
agent state: reflection agent:idle
reflection agent


def min_pal_partition_cuts(s: str) -> int:
    """Return the minimum number of cuts needed to partition the string into palindromic substrings.

    Example: For s = "aab", the minimum cuts is 1 (partition ["aa", "b"]).

    Args:
        s (str): Input string

    Returns:
        int: Minimum cuts needed

    Time Complexity: O(n^2)
    Space Complexity: O(n^2)
    """
    n = len(s)
    is_pal = [[False] * n for _ in range(n)]

    for i in range(n):
        is_pal[i][i] = True

    for i in range(n - 1):
        is_pal[i][i + 1] = (s[i] == s[i + 1])

    for length in range(3, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            if s[i] == s[j] and is_pal[i + 1][j - 1]:
                is_pal[i][j] = True

    dp = [float('inf')] * (n + 1)
    dp[0] = -1

    for i in range(1, n + 1):
        for j in range(i):
  